## Fashion MNIST Classification 문제

### 텐서플로와 다른 라이브러리 임포트

In [ ]:
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# 런타임에서 할당하는데 필요한 양만큼의 GPU 메모리를 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

### 패션 MNIST 데이터셋 임포트하기

load_data() 함수를 호출해서 Fashion MNIST 로딩 : 네 개의 넘파이(NumPy) 배열이 반환

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [6]:
train_images.shape

(60000, 28, 28, 1)

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') #(60000, 28, 28, 1) -> (28,28,1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')

In [7]:
train_images.shape

(60000, 28, 28, 1)

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### 데이터 탐색

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
train_labels

In [ ]:
len(test_labels)

### 데이터 전처리

In [ ]:
plt.figure()
plt.imshow(np.reshape(train_images[0],(28,28))) # channel=1은 받지 않는다 -> imshow함수에서
plt.colorbar()
plt.grid(False)
plt.show()

픽셀 값을 [0,255]에서 [0,1]로 조정

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Traning Set 25개 이미지, 클래스 이름 출력

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.reshape(train_images[i],(28,28)), cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

### 모델 구성

#### Q. keras.layers.Conv2D, MaxPooling2D, Flatten, Dense를 사용해서 모델의 Layer를 구성하시오.
단, 각 계층의 activation은 relu로 구성

In [ ]:
model = keras.Sequential([
      # depth=32, shape는 (3,3), stride는 default 1사용, padding은‘SAME'을 사용해야 pooling시 반으로 줄어든다
      keras.layers.Conv2D(32, (3, 3), padding='SAME', activation='relu', input_shape=(28, 28, 1)),
      keras.layers.BatchNormalization(), #배치 정규화
      keras.layers.MaxPooling2D((2, 2)), #maxpooling은 default로 사용
      keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPooling2D((2, 2)),
      keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPooling2D((2, 2)),
      keras.layers.Flatten(), # (3, 3, 64) 출력을 (576) 크기로 펼침
      keras.layers.Dense(64, activation='relu'),
      keras.layers.Dense(10, activation='softmax') #분류 문제이기 때문에 softmax함수 사용          
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', #분류문제의 loss 설정
              metrics=['accuracy'])

### 모델 훈련

In [ ]:
batch_size = 64
model.fit(train_images, train_labels,
          batch_size=batch_size,
          epochs=5,)

### 테스트 성능 측정 (정확도)

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\n테스트 정확도:', test_acc)

### 예측 만들기

In [ ]:
predictions = model.predict(test_images)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(np.reshape(img,(28,28)), cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# 처음 X 개의 테스트 이미지와 예측 레이블, 진짜 레이블을 출력합니다
# 올바른 예측은 파랑색으로 잘못된 예측은 빨강색으로 나타냅니다
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()